In [32]:
# Imports 
import numpy as np
import random
from datetime import datetime

In [350]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [352]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [3, 75],
 'London': [82, 64],
 'Moscow': [66, 6],
 'Barcelona': [89, 42],
 'Rome': [75, 35],
 'Paris': [33, 15],
 'Vienna': [61, 52],
 'Munich': [67, 36],
 'Istanbul': [55, 56],
 'Kyiv': [18, 69],
 'Bucharest': [67, 91],
 'Minsk': [74, 88],
 'Warsaw': [36, 68],
 'Budapest': [79, 78],
 'Milan': [47, 37],
 'Prague': [55, 77],
 'Sofia': [53, 94],
 'Birmingham': [19, 55],
 'Brussels': [96, 84],
 'Amsterdam': [74, 33]}

In [354]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Amsterdam', 'Rome', 'Prague', ..., 'Birmingham', 'Milan',
        'Kyiv'],
       ['Barcelona', 'Kyiv', 'Prague', ..., 'Birmingham', 'Sofia',
        'Warsaw'],
       ['Rome', 'Budapest', 'Milan', ..., 'Prague', 'Minsk', 'Bucharest'],
       ...,
       ['Warsaw', 'Kyiv', 'Amsterdam', ..., 'Budapest', 'Rome',
        'Brussels'],
       ['Brussels', 'Prague', 'Sofia', ..., 'Vienna', 'Budapest',
        'Bucharest'],
       ['Vienna', 'Bucharest', 'Minsk', ..., 'Warsaw', 'Paris', 'Sofia']],
      dtype='<U10')

In [356]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [358]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([ 719.29907785,  955.96284642,  902.94102363,  912.90252837,
        871.80003823,  781.60704864,  873.34342317,  827.03346895,
        882.1253069 ,  862.01430088,  721.80553106,  853.54871109,
        895.22963425,  890.99069447,  922.31579893,  971.69547051,
        805.30011284,  991.9504098 ,  768.91040645,  970.95571384,
        749.61066242,  904.96881975,  888.56068266,  929.04984337,
        722.80614335,  802.80426711,  804.53705831,  913.89794356,
        874.4343025 ,  824.99112285,  739.58819043,  932.66099745,
        954.38216275,  936.83564121,  934.01040085,  820.08217018,
        893.99566425,  910.204523  ,  947.67430038,  829.38993641,
        945.62822147,  955.85933106,  861.10784711,  954.21072015,
        965.85219284,  755.10480082,  824.79638904,  821.69673013,
        838.7770902 ,  839.1638324 ,  882.51882548,  816.35272866,
        947.08762496,  978.93607447,  840.83328985,  812.64186055,
        762.45692804,  875.49171317,  887.16640484,  949.32968

In [360]:
#selection function
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = (total_fit/fitnes_list)
    prob_list = prob_list/prob_list.sum()
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Sofia', 'Birmingham', 'Amsterdam', 'Budapest', 'Vienna', 'Milan',
       'Barcelona', 'Paris', 'Warsaw', 'Prague', 'Istanbul', 'Rome',
       'Minsk', 'London', 'Kyiv', 'Brussels', 'Moscow', 'Berlin',
       'Munich', 'Bucharest'], dtype='<U10')

In [362]:
#onepoint crossover
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Rome', 'Kyiv', 'Warsaw', 'Sofia', 'Birmingham', 'Munich',
       'Barcelona', 'Brussels', 'Vienna', 'London', 'Bucharest',
       'Budapest', 'Istanbul', 'Minsk', 'Prague', 'Amsterdam', 'Paris',
       'Berlin', 'Moscow', 'Milan'], dtype='<U10')

In [364]:
#mutation by swaping
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['London', 'Kyiv', 'Warsaw', 'Sofia', 'Moscow', 'Munich',
       'Barcelona', 'Budapest', 'Vienna', 'Paris', 'Bucharest',
       'Birmingham', 'Istanbul', 'Minsk', 'Prague', 'Amsterdam',
       'Brussels', 'Berlin', 'Rome', 'Milan'], dtype='<U10')

In [366]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%50==0: print(i, best_solution[1], fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 inf 869.6729931019348 14/10/24 16:07
50 595.8743521581085 871.706064882716 14/10/24 16:07
100 595.8743521581085 865.6802774139281 14/10/24 16:07
150 595.8743521581085 870.9348064477782 14/10/24 16:07
200 513.1901560250673 859.6958676936086 14/10/24 16:07
250 513.1901560250673 865.064767124286 14/10/24 16:07
300 513.1901560250673 871.486459619946 14/10/24 16:07
350 513.1901560250673 865.9792939412446 14/10/24 16:07
400 513.1901560250673 869.849242259403 14/10/24 16:07
450 513.1901560250673 869.2906499533208 14/10/24 16:07
500 513.1901560250673 863.5269163990185 14/10/24 16:07
550 513.1901560250673 880.1042340512349 14/10/24 16:07
600 513.1901560250673 871.2212763810592 14/10/24 16:07
650 513.1901560250673 863.2372546114916 14/10/24 16:07
700 513.1901560250673 873.8937270460074 14/10/24 16:07
750 513.1901560250673 875.6481386781635 14/10/24 16:07
800 513.1901560250673 870.470073639268 14/10/24 16:07
850 513.1901560250673 876.5272029678322 14/10/24 16:07
900 513.1901560250673 866.181610

In [368]:
best_solution

[7496,
 482.004457999315,
 array([['Sofia', 'Prague', 'Kyiv', 'Warsaw', 'Vienna', 'Istanbul',
         'Munich', 'Rome', 'Barcelona', 'Amsterdam', 'London', 'Budapest',
         'Bucharest', 'Brussels', 'Minsk', 'Berlin', 'Birmingham',
         'Milan', 'Paris', 'Moscow']], dtype='<U10')]